<a href="https://colab.research.google.com/github/LeonardoLuca/AskYou/blob/main/rag_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste conexão Gemini


In [1]:
# Célula 1: Instalar as bibliotecas necessárias
!pip install -q langchain langchain-google-genai datasets qdrant-client tiktoken


In [2]:
# Célula 2: Importar bibliotecas e configurar a API Key do Google
import os
import sys
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

# Variável para verificar se a configuração foi bem sucedida
google_api_key_configurada = False
chat = None # Inicializa a variável chat

try:
    # Pega a chave de API dos Secrets do Colab
    # Certifique-se que o nome do secret é 'GOOGLE_API_KEY'
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

    # Verifica se a chave foi realmente obtida (não é None ou vazia)
    if not GOOGLE_API_KEY:
        print("Erro: A chave de API 'GOOGLE_API_KEY' foi encontrada nos Secrets, mas está vazia.", file=sys.stderr)
    else:
        print("Chave de API do Google carregada com sucesso!")
        google_api_key_configurada = True

except userdata.SecretNotFoundError:
    print("Erro: Secret 'GOOGLE_API_KEY' não encontrado.", file=sys.stderr)
    print("Por favor, adicione sua chave de API do Google AI Studio aos Secrets do Colab.", file=sys.stderr)
except Exception as e:
    print(f"Ocorreu um erro inesperado ao buscar a chave de API: {e}", file=sys.stderr)

# Limpa a variável do ambiente se ela existir por algum motivo (boa prática)
# Embora estejamos passando diretamente, é bom garantir que não haja conflito.
if 'GOOGLE_API_KEY' in os.environ:
    del os.environ['GOOGLE_API_KEY']

Chave de API do Google carregada com sucesso!


In [3]:
# Célula 3: Instanciar o modelo Gemini (só executa se a chave foi carregada)

if google_api_key_configurada:
    try:
        # Escolha o modelo Gemini (ex: 'gemini-1.5-pro-latest', 'gemini-1.5-flash-latest')
        chat = ChatGoogleGenerativeAI(
            model="gemini-1.5-pro-latest",  # Ou outro modelo Gemini compatível
            google_api_key=GOOGLE_API_KEY,
            # convert_system_message_to_human=True # Descomente se tiver problemas com SystemMessage
                                                 # Isso faz com que a mensagem do sistema seja anexada
                                                 # à primeira mensagem humana, o que às vezes funciona melhor com Gemini.
        )
        print(f"Modelo {chat.model} instanciado com sucesso.")
    except Exception as e:
        print(f"Erro ao instanciar o modelo ChatGoogleGenerativeAI: {e}", file=sys.stderr)
        chat = None # Garante que chat é None se a instanciação falhar
else:
    print("\nInstanciação do modelo pulada pois a chave de API não foi configurada.", file=sys.stderr)

Modelo models/gemini-1.5-pro-latest instanciado com sucesso.


In [4]:
# Célula 4: Definir as mensagens (geralmente sem alterações)
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="Você é um assistente útil que responde perguntas."),
    HumanMessage(content="Olá Bot, como você está hoje?"),
    AIMessage(content="Estou bem, obrigado por perguntar! Como posso ajudar?"),
    HumanMessage(content="Gostaria de entender o que é Machine Learning")
]

print("Lista de mensagens definida.")

Lista de mensagens definida.


In [5]:
# Célula 5: Invocar o modelo e obter a resposta (com verificação)

res = None # Inicializa a variável de resultado

if chat: # Verifica se o modelo foi instanciado com sucesso
    try:
        print(f"\nEnviando mensagens para o modelo {chat.model}...")
        res = chat.invoke(messages)
        print("Resposta recebida do modelo.")
    except Exception as e:
        print(f"Erro durante a invocação do modelo: {e}", file=sys.stderr)
else:
    print("\nInvocação pulada pois o modelo não foi instanciado corretamente (verifique a API Key e a Célula 3).", file=sys.stderr)

# Exibe o objeto de resposta completo (opcional, útil para debug)
# print("\nObjeto de resposta completo:")
# print(res)


Enviando mensagens para o modelo models/gemini-1.5-pro-latest...
Resposta recebida do modelo.


In [6]:
# Célula 6: Imprimir o conteúdo da resposta (com verificação)

if res: # Verifica se houve uma resposta válida
    print("\n--- Conteúdo da Resposta ---")
    print(res.content)
    print("--------------------------")
elif chat: # Se o chat foi instanciado mas res é None, houve erro na invocação
     print("\nNão foi possível obter uma resposta do modelo (verifique o erro na Célula 5).", file=sys.stderr)
else:
     print("\nNão há resposta para exibir pois o modelo não foi instanciado.", file=sys.stderr)


--- Conteúdo da Resposta ---
Machine Learning (Aprendizado de Máquina) é um subcampo da Inteligência Artificial (IA) que se concentra no desenvolvimento de sistemas que podem aprender com os dados sem serem explicitamente programados. Em outras palavras, ao invés de programar regras específicas para uma tarefa, os algoritmos de Machine Learning aprendem padrões e insights a partir dos dados fornecidos, permitindo-lhes fazer previsões ou tomar decisões.

Imagine, por exemplo, que você queira criar um programa para identificar fotos de gatos. Em vez de codificar manualmente as características de um gato (bigodes, orelhas pontudas, etc.), você alimentaria um algoritmo de Machine Learning com milhares de imagens de gatos e outras imagens que *não* são de gatos. O algoritmo analisaria esses dados, identificaria padrões que distinguem os gatos de outros objetos e aprenderia a reconhecer gatos em novas imagens que nunca viu antes.

**Aqui estão alguns conceitos importantes sobre Machine Lear

In [7]:
messages.append(res)

In [8]:
messages

[SystemMessage(content='Você é um assistente útil que responde perguntas.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Olá Bot, como você está hoje?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Estou bem, obrigado por perguntar! Como posso ajudar?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Gostaria de entender o que é Machine Learning', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Machine Learning (Aprendizado de Máquina) é um subcampo da Inteligência Artificial (IA) que se concentra no desenvolvimento de sistemas que podem aprender com os dados sem serem explicitamente programados. Em outras palavras, ao invés de programar regras específicas para uma tarefa, os algoritmos de Machine Learning aprendem padrões e insights a partir dos dados fornecidos, permitindo-lhes fazer previsões ou tomar decisões.\n\nImagine, por exemplo, que você queira criar um programa para identificar fotos de gatos. Em

In [9]:
prompt = HumanMessage (
    content="Qual a diferença entre supervisionado e não supervisionado?"
    )
messages.append(prompt)

In [10]:
res = chat.invoke(messages)
print(res.content)

A principal diferença entre aprendizado supervisionado e não supervisionado reside na presença ou ausência de **rótulos** nos dados de treinamento:

**Aprendizado Supervisionado:**

* **Dados rotulados:**  Os dados de treinamento incluem tanto as entradas quanto as saídas desejadas (os rótulos). Imagine um professor supervisionando seus alunos, fornecendo as respostas corretas.
* **Objetivo:** Prever a saída para novas entradas com base nos padrões aprendidos a partir dos dados rotulados.  O algoritmo aprende a mapear entradas para saídas.
* **Analogia:** Aprender a identificar frutas com um guia que mostra exemplos de cada fruta e seus respectivos nomes.
* **Exemplos de algoritmos:** Regressão Linear, Regressão Logística, Árvores de Decisão, Support Vector Machines (SVMs), Redes Neurais.
* **Tarefas comuns:** Classificação (prever a categoria de uma entrada, como "gato" ou "cachorro"), Regressão (prever um valor contínuo, como o preço de uma casa).

**Aprendizado Não Supervisionado:**

# RAG

In [38]:
#Conectando com o google drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [14]:
#Importando o dataset do drive
import json

# Path to your JSON file in Google Drive
file_path = '/content/drive/My Drive/Colab Notebooks/datasets/dataset-cv-leonardo.json'

# Load the JSON file
with open(file_path, 'r', encoding='utf-8') as f:
    dataset = json.load(f)

# Display the first entry to verify
print(dataset[0])

{'ID': 1, 'Category': 'Personal Info', 'Subcategory': 'Contact', 'Description': 'Name: Leonardo Luca Araujo Teixeira, Email: luca_leonardo@hotmail.com, Phone: +55 (81) 99133-7728, LinkedIn: www.linkedin.com/in/leonardo-luca-araujo-teixeira-577475130', 'Start_Date': None, 'End_Date': None, 'Location': 'Recife, Pernambuco, Brazil'}


In [17]:
#Bibliotecas para embedding
!pip install -q sentence-transformers faiss-cpu transformers torch

ERROR: Operation cancelled by user


In [18]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Extract descriptions from the dataset
descriptions = [entry['Description'] for entry in dataset]

# Generate embeddings
embeddings = embedder.encode(descriptions, show_progress_bar=True)

# Convert to numpy array for FAISS
embeddings = np.array(embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
print(descriptions[0])
print(embeddings)

Name: Leonardo Luca Araujo Teixeira, Email: luca_leonardo@hotmail.com, Phone: +55 (81) 99133-7728, LinkedIn: www.linkedin.com/in/leonardo-luca-araujo-teixeira-577475130
[[-0.11947705 -0.06836507  0.0472219  ... -0.00589817 -0.11160143
   0.01503693]
 [-0.05834923 -0.01280251 -0.02741665 ... -0.03324935 -0.01748918
   0.04633646]
 [-0.06522979  0.0163146   0.01601382 ... -0.00074401  0.03306707
  -0.03899989]
 ...
 [-0.09192289  0.03410273 -0.02652767 ...  0.07471334  0.02675675
   0.03737945]
 [-0.072395   -0.02435883 -0.01206787 ... -0.0313517  -0.08099234
   0.00353222]
 [-0.09124634 -0.02783696  0.05789485 ... -0.0378456  -0.00637978
  -0.01822451]]


In [40]:
import faiss

# Create a FAISS index
dimension = embeddings.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dimension)  # L2 distance for similarity

# Add embeddings to the index
index.add(embeddings)

# Save the index (optional)
faiss.write_index(index, '/content/drive/My Drive/Colab Notebooks/index/faiss-cv-leonardo-index.bin')

In [41]:
#Load a Generative Model
from transformers import pipeline

# Load a generative model
generator = pipeline('text-generation', model='distilgpt2')

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [44]:
#Implement the RAG Pipeline

def rag_query(query, k=3):
    # Encode the query
    query_embedding = embedder.encode([query])

    # Search for the top-k most similar entries
    distances, indices = index.search(np.array(query_embedding), k)

    # Retrieve corresponding dataset entries
    retrieved_entries = [dataset[idx] for idx in indices[0]]

    # Create a context from retrieved entries
    context = "\n".join([entry['Description'] for entry in retrieved_entries])

    # Generate a response
    prompt = f"Baseado nessa informação a seguir:\n{context}\n\nResponda a questão: {query}"
    response = generator(prompt, max_length=200, num_return_sequences=1)[0]['generated_text']

    return response, retrieved_entries

# Example query
query = "Qual a experiência de Leonardo com Python?"
response, retrieved = rag_query(query)
print("Respostas:", response)
print("Retrieved Entries:", retrieved)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response: Baseado nessa informação a seguir:
Python (Programming Language)
Python Basics
UniCesumar, Postgraduate Specialization in System Development with Python (January 2024 - December 2024).

Responda a questão: Qual a experiência de Leonardo com Python?
Arts:
Python, Java, PHP, Oracle and Python are the only Python programs which use the Python language and the tools available.
Dependency for Python
python2.0 (1) of JIT.
Python2.0 (1) of Apache/Slovenia (1) of Python2.0 (1) of
Python3.0 (1) of Apache/Slovenia (1) of Python2.0 (1) of Python2.0 (1) of Python2.0 (1) of Python2.0 (1) of Python2.0 (1) of Python
Retrieved Entries: [{'ID': 4, 'Category': 'Skills', 'Subcategory': 'Technical', 'Description': 'Python (Programming Language)', 'Start_Date': None, 'End_Date': None, 'Location': None}, {'ID': 11, 'Category': 'Certifications', 'Subcategory': 'Certification', 'Description': 'Python Basics', 'Start_Date': None, 'End_Date': None, 'Location': None}, {'ID': 25, 'Category': 'Education'